# Linear Regression using Dask Data Frames

This post includes code from [Scalable-Data-Analysis-in-Python-with-Dask](https://github.com/PacktPublishing/-Scalable-Data-Analysis-in-Python-with-Dask/tree/master/Section%202) and [coiled-examples](https://github.com/coiled/coiled-examples).

In [28]:
import numpy as np
import dask.array as da
import pandas as pd
import sqlalchemy as db
from sqlalchemy import create_engine
import sqlite3
import pandas as pd

In [29]:
# engine = db.create_engine("sqlite:///fiscal.db")
# connection = engine.connect()
# metadata = db.MetaData()

In [30]:
#engine.execute("SELECT * FROM fiscal_data LIMIT 1").fetchall()

In [31]:
# sql = """
# SELECT year
# , region
# , province
# , gdp
# , fdi
# , it
# , specific
# FROM fiscal_table
# """
#
# cnxn = connection

In [32]:
#df = pd.read_sql(sql, cnxn)

In [72]:
import pandas as pd
url = 'https://raw.githubusercontent.com/davidrkearney/Kearney_Data_Science/master/_notebooks/df_panel_fix.csv'
df = pd.read_csv(url, error_bad_lines=False)
df

df = df[['year', 'province', 'gdp', 'fdi', 'it', 'specific']]
df

/tmp/ipykernel_191754/2939603070.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(url, error_bad_lines=False)


,year,province,gdp,fdi,it,specific
0,1996,Anhui,2093.30,50661,631930,147002.0
1,1997,Anhui,2347.32,43443,657860,151981.0
2,1998,Anhui,2542.96,27673,889463,174930.0
3,1999,Anhui,2712.34,26131,1227364,285324.0
4,2000,Anhui,2902.09,31847,1499110,195580.0
...,...,...,...,...,...,...
355,2003,Zhejiang,9705.02,498055,2261631,391292.0
356,2004,Zhejiang,11648.70,668128,3162299,656175.0
357,2005,Zhejiang,13417.68,772000,2370200,656175.0
358,2006,Zhejiang,15718.47,888935,2553268,1017303.0


In [35]:
from dask.distributed import Client

client = Client(processes=False, threads_per_worker=2,
                n_workers=3, memory_limit='4GB')
client

/home/gao/anaconda3/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44929 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.0.198:44929/status,
Dashboard: http://192.168.0.198:44929/status,Workers: 3
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: False
Comm: inproc://192.168.0.198/191754/19,Workers: 3
Dashboard: http://192.168.0.198:44929/status,Total threads: 6
Started: Just now,Total memory: 11.18 GiB
Comm: inproc://192.168.0.198/191754/23,Total threads: 2
Dashboard: http://192.168.0.198:45683/status,Memory: 3.73 GiB
Nanny: None,


In [36]:
client.restart()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.0.198:44929/status,
Dashboard: http://192.168.0.198:44929/status,Workers: 3
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: False
Comm: inproc://192.168.0.198/191754/19,Workers: 3
Dashboard: http://192.168.0.198:44929/status,Total threads: 6
Started: Just now,Total memory: 11.18 GiB
Comm: inproc://192.168.0.198/191754/23,Total threads: 2
Dashboard: http://192.168.0.198:45683/status,Memory: 3.73 GiB
Nanny: None,


In [37]:
from dask import dataframe as dd 

In [38]:
ddf = dd.from_pandas(df, npartitions=5)

In [39]:
print(ddf)

Dask DataFrame Structure:
                year province      gdp    fdi     it specific
npartitions=5                                                
0              int64   object  float64  int64  int64  float64
72               ...      ...      ...    ...    ...      ...
...              ...      ...      ...    ...    ...      ...
288              ...      ...      ...    ...    ...      ...
359              ...      ...      ...    ...    ...      ...
Dask Name: from_pandas, 5 tasks


In [40]:
ddf.head()

,year,province,gdp,fdi,it,specific
0,1996,Anhui,2093.30,50661,631930,147002.0
1,1997,Anhui,2347.32,43443,657860,151981.0
2,1998,Anhui,2542.96,27673,889463,174930.0
3,1999,Anhui,2712.34,26131,1227364,285324.0
4,2000,Anhui,2902.09,31847,1499110,195580.0


In [41]:
client.id

'Client-d0860491-426d-11ed-ad0a-6d51532fdfa7'

## Selecting Features and Target

In [46]:
feat_list = ["year", "fdi"]
cat_feat_list = ["province"]
target = ["gdp"]

In [47]:
ddf["year"] = ddf["year"].astype(int)
ddf["fdi"] = ddf["fdi"].astype(float)
ddf["gdp"] = ddf["gdp"].astype(float)
ddf["it"] = ddf["it"].astype(float)

In [48]:
#OHE
from dask_ml.preprocessing import OneHotEncoder

In [49]:
ddf = ddf.categorize(cat_feat_list)

In [50]:
ohe = OneHotEncoder(sparse=False)

In [51]:
ohe_ddf = ohe.fit_transform(ddf[cat_feat_list])

In [52]:
feat_list = feat_list + ohe_ddf.columns.tolist()
feat_list = [f for f in feat_list if f not in cat_feat_list]

In [53]:
ddf_processed = (dd.concat([ddf,ohe_ddf], axis=1) [feat_list + target])

In [54]:
ddf_processed.compute()

,year,fdi,province_Anhui,province_Beijing,province_Chongqing,province_Fujian,province_Gansu,province_Guangdong,province_Guangxi,province_Guizhou,...,province_Shandong,province_Shanghai,province_Shanxi,province_Sichuan,province_Tianjin,province_Tibet,province_Xinjiang,province_Yunnan,province_Zhejiang,gdp
0,1996,50661.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2093.30
1,1997,43443.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2347.32
2,1998,27673.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2542.96
3,1999,26131.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2712.34
4,2000,31847.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2902.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2003,498055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9705.02
356,2004,668128.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11648.70
357,2005,772000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13417.68
358,2006,888935.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15718.47


In [55]:
feat_list

['year',
 'fdi',
 'province_Anhui',
 'province_Beijing',
 'province_Chongqing',
 'province_Fujian',
 'province_Gansu',
 'province_Guangdong',
 'province_Guangxi',
 'province_Guizhou',
 'province_Hainan',
 'province_Hebei',
 'province_Heilongjiang',
 'province_Henan',
 'province_Hubei',
 'province_Hunan',
 'province_Jiangsu',
 'province_Jiangxi',
 'province_Jilin',
 'province_Liaoning',
 'province_Ningxia',
 'province_Qinghai',
 'province_Shaanxi',
 'province_Shandong',
 'province_Shanghai',
 'province_Shanxi',
 'province_Sichuan',
 'province_Tianjin',
 'province_Tibet',
 'province_Xinjiang',
 'province_Yunnan',
 'province_Zhejiang']

## Dask Linear Regression

In [56]:
X=ddf_processed[feat_list].persist()
y=ddf_processed[target].persist()

In [57]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge

In [58]:
X

,year,fdi,province_Anhui,province_Beijing,province_Chongqing,province_Fujian,province_Gansu,province_Guangdong,province_Guangxi,province_Guizhou,province_Hainan,province_Hebei,province_Heilongjiang,province_Henan,province_Hubei,province_Hunan,province_Jiangsu,province_Jiangxi,province_Jilin,province_Liaoning,province_Ningxia,province_Qinghai,province_Shaanxi,province_Shandong,province_Shanghai,province_Shanxi,province_Sichuan,province_Tianjin,province_Tibet,province_Xinjiang,province_Yunnan,province_Zhejiang
npartitions=5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
72,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [59]:
X.compute()

,year,fdi,province_Anhui,province_Beijing,province_Chongqing,province_Fujian,province_Gansu,province_Guangdong,province_Guangxi,province_Guizhou,...,province_Shaanxi,province_Shandong,province_Shanghai,province_Shanxi,province_Sichuan,province_Tianjin,province_Tibet,province_Xinjiang,province_Yunnan,province_Zhejiang
0,1996,50661.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1997,43443.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1998,27673.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1999,26131.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2000,31847.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2003,498055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
356,2004,668128.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
357,2005,772000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
358,2006,888935.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [60]:
y

,gdp
npartitions=5,
0,float64
72,...
...,...
288,...
359,...


In [61]:
LinReg = LinearRegression()

In [62]:
LinReg.fit(X, y)

LinearRegression()

In [65]:
RidgeReg = Ridge()
RidgeReg.fit(X, y)

Ridge()

In [67]:
LinReg.predict(X)[:5]

array([[1830.87851075],
       [2076.99855131],
       [2220.28956049],
       [2534.65768128],
       [2936.29581023]])

In [69]:
RidgeReg.predict(X)[:5]

array([[1743.12844255],
       [1993.63521868],
       [2142.78090348],
       [2460.56148157],
       [2864.36624124]])

In [70]:
client.restart()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.0.198:44929/status,
Dashboard: http://192.168.0.198:44929/status,Workers: 3
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: False
Comm: inproc://192.168.0.198/191754/19,Workers: 3
Dashboard: http://192.168.0.198:44929/status,Total threads: 6
Started: Just now,Total memory: 11.18 GiB
Comm: inproc://192.168.0.198/191754/23,Total threads: 2
Dashboard: http://192.168.0.198:45683/status,Memory: 3.73 GiB
Nanny: None,


In [71]:
client.close()